<a href="https://colab.research.google.com/github/mshilpaa/ethereum-data-analysis/blob/master/Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Out going transactions:
```
create table `civic-bruin-270815.ethereum.out_transaction` as (SELECT from_address as address, count(*) as no_of_out_transactions, sum(value) as tot_ether_sent,'OUT' as tax_type FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE from_address in (select address from `bigquery-public-data.crypto_ethereum.balances` LIMIT 400000) group by from_address) 
```

Incoiming transactions
```
create table `civic-bruin-270815.ethereum.in_transaction` as (SELECT to_address as address, count(*) as no_of_in_transactions, sum(value) as tot_ether_recieved,'IN' as tax_type FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE to_address in (select address from `civic-bruin-270815.ethereum.in_transaction` LIMIT 2290371) group by to_address) 
```

join the two tables
```
create table `civic-bruin-270815.ethereum.in_out_transaction` as (SELECT out_txn.address, out_txn.no_of_out_transactions, out_txn.tot_ether_sent, in_txn.no_of_in_transactions, in_txn.tot_ether_recieved from `civic-bruin-270815.ethereum.in_transaction` as in_txn inner join `civic-bruin-270815.ethereum.out_transaction` as out_txn on in_txn.address = out_txn.address) 
```

tokens recieved
```
SELECT to_address as address, count(distinct token_address) as tot_distinct_tokens_recieved, count(*) as tot_tokens_recieved, sum(safe_cast(value as FLOAT64)) as tot_token_value_recieved FROM `bigquery-public-data.crypto_ethereum.token_transfers` WHERE to_address in ( select address from `civic-bruin-270815.ethereum.in_out_transaction`) group by to_address
```

tokens sent
```
SELECT from_address as address, count(distinct token_address) as tot_distinct_tokens_sent, count(*) as tot_tokens_sent, sum(safe_cast(value as FLOAT64)) as tot_token_value_sent FROM `bigquery-public-data.crypto_ethereum.token_transfers` WHERE from_address in ( select address from `civic-bruin-270815.ethereum.in_out_transaction`) group by from_address 
```